In [ ]:
#Loading in Packages and Data

#Importing Packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr
import os; import time
import pickle
import h5py
###############################################################
def coefs(coefficients,degree):
    coef=coefficients
    coefs=""
    for n in range(degree, -1, -1):
        string=f"({coefficients[len(coef)-(n+1)]:.1e})"
        coefs+=string + f"x^{n}"
        if n != 0:
            coefs+=" + "
    return coefs
###############################################################

# Importing Model Data
check=False
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'

# # dx = 1 km; Np = 1M; Nt = 5 min
# data=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_1km_5min.nc', decode_timedelta=True) #***
# parcel=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_pdata_1km_5min_1e6.nc', decode_timedelta=True) #***
# res='1km';t_res='5min'
# Np_str='1e6'

# # dx = 1km; Np = 50M
# #Importing Model Data
# dir2='/home/air673/koa_scratch/'
# data=xr.open_dataset(dir2+'cm1out_1km_1min.nc', decode_timedelta=True) #***
# parcel=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_50M.nc', decode_timedelta=True) #***
# res='1km'; t_res='1min'; Np_str='50e6'

# # dx = 1km; Np = 50M; Nz = 95
# #Importing Model Data
# dir2='/home/air673/koa_scratch/'
# data=xr.open_dataset(dir2+'cm1out_1km_1min_95nz.nc', decode_timedelta=True) #***
# parcel=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_95nz.nc', decode_timedelta=True) #***
# res='1km'; t_res='1min_95nz'; Np_str='50e6'

# dx = 250m; Np = 50M
#Importing Model Data
dir2='/home/air673/koa_scratch/'
data=xr.open_dataset(dir2+'cm1out_250m_1min_50M.nc', decode_timedelta=True) #***
parcel=xr.open_dataset(dir2+'cm1out_pdata_250m_1min_50M.nc', decode_timedelta=True) #***
res='250m'; t_res='1min'; Np_str='50e6'

In [ ]:
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
path=dir2+'../Functions/'
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions

# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

#####

#Import StatisticalFunctions 
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
path=dir2+'../Functions/'
sys.path.append(path)

import StatisticalFunctions
from StatisticalFunctions import * # import NumericalFunctions 

In [ ]:
##########################################################################################
#Plotting

In [ ]:
def averaged_profiles(profile):
    out_var=profile[ (profile[:, 1] > 1)]; #gets rid of rows that have no data\n"
    out_var=np.array([out_var[:, 0] / out_var[:, 1], out_var[:, 2]]).T #divides the data column by the counter column
    return out_var

In [ ]:
def LoadAllCloudBase():
    dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/'
    in_file = dir2 + f"all_cloudbase_{res}_{t_res}_{Np_str}.pkl"
    with open(in_file, 'rb') as f:
        all_cloudbase = pickle.load(f)
    return(all_cloudbase)
min_all_cloudbase=np.nanmin(LoadAllCloudBase())
cloudbase=min_all_cloudbase
print(f"Minimum Cloudbase is: {cloudbase}\n")

def LoadMeanLFC():
    dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/'
    in_file = dir2 + f"MeanLFC_{res}_{t_res}_{Np_str}.pkl"
    with open(in_file, 'rb') as f:
        MeanLFC = pickle.load(f)
    return MeanLFC
MeanLFC=LoadMeanLFC()
print(f"Mean LFC is: {MeanLFC}\n")

In [ ]:
#thresholds
w_thresh1=0.1
w_thresh2=0.5
qcqi_thresh=1e-6

In [ ]:
def load_and_prepare(type_):
    dir2 = dir + 'Project_Algorithms/Domain_Profiles/'
    if type_ == "general":
        fname = f'job_out/general_eulerian_profiles_{res}_{t_res}_{Np_str}.h5'
    else:
        fname = f'job_out/cloudy_eulerian_profiles_{res}_{t_res}_{Np_str}.h5'

    with h5py.File(dir2 + fname, 'r') as f:
        th = np.array(f["profile_th"])
        th_v = np.array(f["profile_th_v"])
        th_e = np.array(f["profile_th_e"])

    # def prep(p):
    #     return np.column_stack([p[:,0]/p[:,1], p[:,2]])
    def prep(f):
        # ratio = np.empty_like(f[:,0])
        # np.divide(f[:,0], f[:,1], out=ratio, where=f[:,1]>0)
        # ratio[f[:,1]==0] = np.nan  # explicitly set where denom=0
        # return np.column_stack([ratio, f[:,2]])
        return averaged_profiles(f)
    return prep(th), prep(th_v), prep(th_e)

In [ ]:
def load_all_profiles():
    data = {}
    for typ in ['cloudy', 'general']:
        th, th_v, th_e = load_and_prepare(typ)
        data[typ] = {'th': th, 'th_v': th_v, 'th_e': th_e}
    return data

def compute_all_derivatives(profiles):
    derivs = {}
    for typ in profiles:
        derivs[typ] = {
            'th': Profile_Ddz(profiles[typ]['th']),
            'th_v': Profile_Ddz(profiles[typ]['th_v']),
            'th_e': Profile_Ddz(profiles[typ]['th_e'])
        }
    return derivs
def plot_profiles_and_derivs_dict_1(profiles, derivs, cloudbase):
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    fig.suptitle(
        f"Average General ($w_{{thresh}}$ ≥ {w_thresh1}, $q_c+q_i$ < {qcqi_thresh}) vs "
        f"Cloudy ($w_{{thresh}}$ ≥ {w_thresh2}, $q_c+q_i$ ≥ {qcqi_thresh})"
    )

    # Panel 1: θ and θₑ
    ax = axes[0]
    styles = {'cloudy': 'b', 'general': 'k'}
    for typ in profiles:
        ax.plot(profiles[typ]['th'][:,0], profiles[typ]['th'][:,1], styles[typ] + '--', label=fr'$\theta$ ({typ})')
        ax.plot(profiles[typ]['th_e'][:,0], profiles[typ]['th_e'][:,1], styles[typ] + '-', label=fr'$\theta_e$ ({typ})')
    
    ax.axhline(cloudbase, color='purple', linestyle='dashed')
    ax.axhline(MeanLFC / 1000, color='green', linestyle='dashed')
    ax.set(xlabel='(K)', ylabel='z (km)', ylim=(0, None))
    ax.legend(loc='lower right')
    ax.grid(True)

    # Panel 2: dθ/dz and dθₑ/dz
    ax = axes[1]
    for typ in derivs:
        ax.plot(derivs[typ]['th'][:,0], derivs[typ]['th'][:,1], styles[typ] + '--', label=fr'$d\theta/dz$ ({typ})')
        ax.plot(derivs[typ]['th_e'][:,0], derivs[typ]['th_e'][:,1], styles[typ] + '-', label=fr'$d\theta_e/dz$ ({typ})')

    ax.axvline(0, color='gray', linestyle='--', alpha=0.5)
    ax.axhline(cloudbase, color='purple', linestyle='dashed')
    ax.axhline(MeanLFC / 1000, color='green', linestyle='dashed')
    ax.set(xlabel='(K/km)', ylabel='z (km)', ylim=(0, None))
    ax.legend(loc='lower right')
    ax.grid(True)

    plt.tight_layout()
    plt.show()

def plot_profiles_and_derivs_dict_2(profiles, derivs, cloudbase):
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    fig.suptitle(
        f"Average General ($w_{{thresh}}$ ≥ {w_thresh1}, $q_c+q_i$ < {qcqi_thresh}) vs "
        f"Cloudy ($w_{{thresh}}$ ≥ {w_thresh2}, $q_c+q_i$ ≥ {qcqi_thresh})"
    )

    # Panel 1: θ and θₑ
    ax = axes[0]
    styles = {'cloudy': 'b', 'general': 'k'}
    for typ in profiles:
        ax.plot(profiles[typ]['th'][:,0], profiles[typ]['th'][:,1], styles[typ] + '--', label=fr'$\theta$ ({typ})')
        ax.plot(profiles[typ]['th_v'][:,0], profiles[typ]['th_v'][:,1], styles[typ] + '-', label=fr'$\theta_v$ ({typ})')
    
    ax.axhline(cloudbase, color='purple', linestyle='dashed')
    ax.axhline(MeanLFC / 1000, color='green', linestyle='dashed')
    ax.set(xlabel='(K)', ylabel='z (km)', ylim=(0, None))
    ax.legend(loc='lower right')
    ax.grid(True)

    # Panel 2: dθ/dz and dθₑ/dz
    ax = axes[1]
    for typ in derivs:
        ax.plot(derivs[typ]['th'][:,0], derivs[typ]['th'][:,1], styles[typ] + '--', label=fr'$d\theta/dz$ ({typ})')
        ax.plot(derivs[typ]['th_v'][:,0], derivs[typ]['th_v'][:,1], styles[typ] + '-', label=fr'$d\theta_v/dz$ ({typ})')

    ax.axvline(0, color='gray', linestyle='--', alpha=0.5)
    ax.axhline(cloudbase, color='purple', linestyle='dashed')
    ax.axhline(MeanLFC / 1000, color='green', linestyle='dashed')
    ax.set(xlabel='(K/km)', ylabel='z (km)', ylim=(0, None))
    ax.legend(loc='lower right')
    ax.grid(True)

    plt.tight_layout()
    plt.show()

In [ ]:
profiles = load_all_profiles()
derivs = compute_all_derivatives(profiles)
plot_profiles_and_derivs_dict_1(profiles, derivs, cloudbase)

In [ ]:
profiles = load_all_profiles()
derivs = compute_all_derivatives(profiles)
plot_profiles_and_derivs_dict_2(profiles, derivs, cloudbase)

In [ ]:
###############################################################
#PLOTTING DWDZ

In [ ]:
def load_and_prepare2(type_):
    dir2 = dir + 'Project_Algorithms/Domain_Profiles/'
    if type_ == "general":
        fname = f'job_out/general_eulerian_profiles_{res}_{t_res}_{Np_str}.h5'
    else:
        fname = f'job_out/cloudy_eulerian_profiles_{res}_{t_res}_{Np_str}.h5'

    with h5py.File(dir2 + fname, 'r') as f:
        w = np.array(f["profile_w"])

    # def prep(p):
    #     return np.column_stack([p[:,0]/p[:,1], p[:,2]])
    def prep(f):
        # ratio = np.empty_like(f[:,0])
        # np.divide(f[:,0], f[:,1], out=ratio, where=f[:,1]>0)
        # ratio[f[:,1]==0] = np.nan  # explicitly set where denom=0
        # return np.column_stack([ratio, f[:,2]])
        return averaged_profiles(f)
    return prep(w)

def load_all_profiles2():
    data = {}
    for typ in ['cloudy', 'general']:
        w = load_and_prepare2(typ)
        data[typ] = {'w': w}
    return data

def compute_all_derivatives2(profiles):
    derivs = {}
    for typ in profiles:
        derivs[typ] = {
            'w': Profile_Ddz(profiles[typ]['w'])
        }
    return derivs
def plot_profiles_and_derivs_dict_3(profiles, derivs, cloudbase):
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    fig.suptitle(
        f"Average General ($w_{{thresh}}$ ≥ {w_thresh1}, $q_c+q_i$ < {qcqi_thresh}) vs "
        f"Cloudy ($w_{{thresh}}$ ≥ {w_thresh2}, $q_c+q_i$ ≥ {qcqi_thresh})"
    )

    # Panel 1: θ and θₑ
    ax = axes[0]
    styles = {'cloudy': 'b', 'general': 'k'}
    for typ in profiles:
        ax.plot(profiles[typ]['w'][:,0], profiles[typ]['w'][:,1], styles[typ] + '-', label=fr'$w$ ({typ})')
    
    ax.axhline(cloudbase, color='purple', linestyle='dashed')
    ax.axhline(MeanLFC / 1000, color='green', linestyle='dashed')
    ax.set(xlabel=r"$\mathrm{m\ s^{-1}}$", ylabel='z (km)', ylim=(0, None))
    ax.legend(loc='upper right')
    ax.grid(True)

    # Panel 2: dθ/dz and dθₑ/dz
    ax = axes[1]
    for typ in derivs:
        ax.plot(derivs[typ]['w'][:,0], derivs[typ]['w'][:,1], styles[typ] + '-', label=fr'$dw/dz$ ({typ})')

    ax.axvline(0, color='gray', linestyle='--', alpha=0.5)
    ax.axhline(cloudbase, color='purple', linestyle='dashed')
    ax.axhline(MeanLFC / 1000, color='green', linestyle='dashed')
    ax.set(xlabel=r"$\mathrm{m\ s^{-1}\ km^{-1}}$", ylabel='z (km)', ylim=(0, None))
    ax.legend(loc='upper right')
    ax.grid(True)

    plt.tight_layout()
    plt.show()


In [ ]:
profiles = load_all_profiles2()
derivs = compute_all_derivatives2(profiles)
plot_profiles_and_derivs_dict_3(profiles, derivs, cloudbase)